In [53]:
import numpy as np

import gc
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import *
from tensorflow.python.keras.layers import Layer
from tensorflow.keras import regularizers

from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

from tensorflow.keras import optimizers,initializers
from tensorflow.python.keras.initializers import glorot_normal
import joblib
import json

from tensorflow.keras import layers, Model
import utils
from utils import *
import importlib
importlib.reload(utils)

from features import *
from tensorflow.keras.callbacks import LambdaCallback

import models
from models import *
importlib.reload(models)
from tensorflow.keras.optimizers import Adam


In [2]:
train = joblib.load('./data_and_feature/train.txt')
val = joblib.load('./data_and_feature/val.txt')
test = joblib.load('./data_and_feature/test.txt')
encoder = joblib.load('./data_and_feature/encoder.txt')

train_num = len(train)

In [54]:
embedding_feat_dict=read_json_file('/Users/bytedance/Desktop/wechat_MTL/config/embedding_feat_dict.json')

In [49]:
# 构建输入数据
train_model_input = {name: train[name] if name not in varlen_features else np.stack(train[name]) for name in feature_names } #训练模型的输入，字典类型。名称和具体值
val_model_input = {name: val[name] if name not in varlen_features else np.stack(val[name]) for name in feature_names }
test_model_input = {name: test[name] if name not in varlen_features else np.stack(test[name]) for name in feature_names}

train_labels = [train[y].values for y in target]
val_labels = [val[y].values for y in target]



In [55]:
sparse_features = embedding_feat_dict['sparse']

sequence_features = embedding_feat_dict['sequence']

dense_features = embedding_feat_dict['dense']

task_names = target

In [56]:
model = MMoE_model(dense_features, sparse_features, varlen_features, encoder, task_names, embedding_feat_dict)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss={task: 'binary_crossentropy' for task in target},
    metrics={task: ['AUC'] for task in target}
)

model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ userid (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feedid (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ authorid            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bgm_song_id         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bgm_singer_id       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ manual_tag_list     │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ manual_keyword_list │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ videoplayseconds    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_userid          │ (None, 1, 6)      │    120,006 │ userid[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_feedid          │ (None, 1, 6)      │    595,032 │ feedid[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_authorid        │ (None, 1, 6)      │    111,744 │ authorid[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_bgm_song_id     │ (None, 1, 6)      │    142,440 │ bgm_song_id[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_bgm_singer_id   │ (None, 1, 6)      │     99,618 │ bgm_singer_id[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_manual_tag_list │ (None, 11, 6)     │      2,100 │ manual_tag_list[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_manual_keyword… │ (None, 18, 6)     │    132,870 │ manual_keyword_l… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_67          │ (None, 1)         │          0 │ videoplayseconds… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_68          │ (None, 6)         │          0 │ emb_userid[0][0]  │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,242,918 (4.74 MB)

 Trainable params: 1,242,918 (4.74 MB)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.fit(
    train_model_input,
    train_labels,
    batch_size=256,
    epochs=1,
    validation_split=0.1,
    verbose=1)

23586/23586 ━━━━━━━━━━━━━━━━━━━━ 124s 5ms/step - click_avatar_AUC: 0.6976 - click_avatar_loss: 0.0484 - forward_AUC: 0.6780 - forward_loss: 0.0314 - like_AUC: 0.7634 - like_loss: 0.1098 - loss: 0.3017 - read_comment_AUC: 0.8735 - read_comment_loss: 0.1120 - val_click_avatar_AUC: 0.8186 - val_click_avatar_loss: 0.0363 - val_forward_AUC: 0.8327 - val_forward_loss: 0.0201 - val_like_AUC: 0.8556 - val_like_loss: 0.0896 - val_loss: 0.2365 - val_read_comment_AUC: 0.9347 - val_read_comment_loss: 0.0906
